In [1]:
import pandas as pd
import seaborn as sns
import glob
from numpy import genfromtxt
# from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import numpy as np
import json
from collections import OrderedDict
import os
import re
import logging
import multiprocessing
from functools import partial
from datetime import datetime
from Bio import SeqIO
from Bio.Seq import Seq
import gzip
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html

In [2]:
### panta input directory (prokka)
pantain_dirdb = 'data/prokka/Ecoli1936/prokka_train/train1/'
pantain_dir = 'data/prokka/Ecoli1936/prokka_test/test1/'
### panta output directory
pantaout_dirdb = 'data/pantaOut/pantatrain1/'
pantaout_dir = 'data/pantaOut/pantatest1/'

### panta input directory (prokka)
# pantain_dirdb = 'data/Ecoli1936/prokka_train/train1/'
# pantain_dir = 'data/Ecoli1936/prokka_test/test1/'
# ### panta output directory
# pantaout_dirdb = 'output/pantatrain1/'
# pantaout_dir = 'output/pantatest1/'

## Find correspondence (injection) from db-database (train set) clusters to test clusters
#### use the representative genes

In [3]:
# find representative gene for each cluster
with open(pantaout_dir + 'annotated_clusters.json', 'r') as JSON:
    json_dict = json.load(JSON)
    
cluster2representativegenedict = {}
representativegene2clusterdict = {}
for key in json_dict:
    cluster2representativegenedict[key] = json_dict[key]['representative']
    representativegene2clusterdict[json_dict[key]['representative']] = key

In [4]:
with open(pantaout_dirdb + 'annotated_clusters.json', 'r') as JSON:
    json_dictdb = json.load(JSON)

cluster2representativegenedictdb = {}
representativegenedict2clusterdb = {}
for key in json_dictdb:
    cluster2representativegenedictdb[key] = json_dictdb[key]['representative']
    representativegenedict2clusterdb[json_dictdb[key]['representative']] = key

In [5]:
# for key in json_dict:
#     print(key, json_dict[key])
#     break;

In [6]:
## PA matrix of db
pa_matrixdb = pd.read_csv(pantaout_dirdb+'gene_presence_absence.Rtab', sep='\t', index_col=0).T
n_samples = pa_matrixdb.shape[0]
n_genes = pa_matrixdb.shape[1]
colsumdb = pa_matrixdb.sum()
core_gene_clusterdb = [colsumdb.index[idx] for idx in range(n_genes) if colsumdb[idx] >= 0.9999*n_samples] # E. coli

In [7]:
## PA matrix of new samples
pa_matrix = pd.read_csv(pantaout_dir+'gene_presence_absence.Rtab', sep='\t', index_col=0).T

In [8]:
clusters = list(pa_matrix.columns)
clustersdb = list(pa_matrixdb.columns)

In [9]:
clustersdb2clustersdict = {}
for idx in range(len(clustersdb)):
    # clustersdb[idx]
    repidxdb = cluster2representativegenedictdb[clustersdb[idx]]
    if  repidxdb in representativegene2clusterdict:
        correspond_cluster = representativegene2clusterdict[repidxdb]
        clustersdb2clustersdict[clustersdb[idx]] = correspond_cluster
    else:
        for key in json_dict:
            if repidxdb in json_dict[key]['gene_id']:
                correspond_cluster = key
                clustersdb2clustersdict[clustersdb[idx]] = correspond_cluster
                break;

In [10]:
# change the order of presence and absence matrix
dbclusterindex = [clustersdb2clustersdict[key] for key in clustersdb]
# PA matrix of test set
pa_matrixnew = pa_matrix[dbclusterindex]

In [11]:
# Note that the cluster name can be different but they share the rep gene

# Feature engineering

## Find all AMR clusters (db)

In [14]:
from pangraph.utils import parse_gff_AMRgene_finder

In [15]:
amr_gene = []
for data_dir in glob.glob(pantain_dirdb + '*.gff'):
    # print(data_dir)
    in_fh = open(data_dir)
    sample_id = data_dir.split('/')[-1][:-4]
    amr_gene += parse_gff_AMRgene_finder(in_fh, sample_id)
    in_fh.close()

In [16]:
# amr_gene[:3], len(amr_gene)

In [17]:
## Create map from gene ID to cluster ID (db)
with open(pantaout_dirdb + 'annotated_clusters.json', 'r') as JSON:
    json_dictdb = json.load(JSON)

gene2clusterdictdb = {}
for key in json_dictdb:
    if len(json_dictdb[key])==0:
        gene2clusterdictdb[key] = key
    for gene in json_dictdb[key]['gene_id']:
        gene2clusterdictdb[gene] = key

In [18]:
#### Map genes back to cluster IDs
amr_clusterID = [gene2clusterdictdb[gene] for gene in amr_gene]
amr_clusterID = list(set(amr_clusterID))

In [19]:
len(amr_clusterID), amr_clusterID[0:4]

(216, ['catA2', 'mph_A_', 'alsC', 'glnQ'])

## Compute K-mer of AMR clusters (db)

In [20]:
from pangraph.utils import binary_label
from sklearn.feature_selection import mutual_info_classif, chi2

In [21]:
# # Read prepresentative sequence
# from Bio import SeqIO
# genecluster2representativeseq = {}
# with open(pantaout_dirDB+'representative_clusters_prot.fasta') as handle:
#     for record in SeqIO.parse(handle, "fasta"):
#         name, sequence = record.id, str(record.seq)
#         genecluster2representativeseq[name] = sequence
#         # print(name,'----', sequence)

In [22]:
with open(pantaout_dirdb + 'samples.json', 'r') as JSON:
    sample_dictdb = json.load(JSON)
sample2integerindexdb = {}
for idx in range(len(sample_dictdb)):
    sample2integerindexdb[sample_dictdb[idx]['id']] = idx
n_samplesdb = len(sample_dictdb)

In [23]:
computed_gene_cluster = amr_clusterID;

In [24]:
# amr_mat = None;
ksize = 10; # k = 10 for protein, 20 for DNA
kmer_list = [];
pairdata = []
for idx in range(len(computed_gene_cluster)):
    alignment_dir = pantaout_dirdb + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    # alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.fna.aln.gz'
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            seqraw = sequence.replace('-', '')
            n_kmers = len(seqraw) - ksize + 1
            for i in range(n_kmers):
                kmer = seqraw[i:i + ksize] 
                # kmer = computed_gene_cluster[idx] + seqraw[i:i + ksize] # compute unique kmer for eaach cluster
                kmer_list.append(kmer)
                pairdata.append((sample2integerindexdb[sample_id], kmer))

In [25]:
unique_kmer = list(set(kmer_list))

In [26]:
kmer2index = {}
for i in range(len(unique_kmer)):
    kmer2index[unique_kmer[i]] = i

In [27]:
kmer_matrix = np.zeros((n_samplesdb, len(unique_kmer)))

In [28]:
# ct = 0
for u, v in pairdata:
    # kmer_matrix[u, kmer2index[v]] += 1
    kmer_matrix[u, kmer2index[v]] = 1

In [29]:
kmer_matrix.shape

(1248, 188685)

In [30]:
# selector = VarianceThreshold(threshold=0.05)
selector = VarianceThreshold(threshold=0.01)
kmer_matrix_VT = selector.fit_transform(kmer_matrix)

In [31]:
# (1653, 59580)
kmer_matrix_VT.shape

(1248, 66461)

In [33]:
kmerindexdb = np.array(unique_kmer)[selector.get_support()==True]

In [35]:
kmer_matrixdbdf = pd.DataFrame(kmer_matrix_VT, columns = kmerindexdb)

In [36]:
kmer_matrix_VT, kmer_matrix_VT.shape, kmerindexdb, kmerindexdb.shape

(array([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 1., 0.],
        ...,
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (1248, 66461),
 array(['NKATLIVEDD', 'PILMGFWIRG', 'VAKSLITMVQ', ..., 'GKSLKWGLNH',
        'TANNHADFAR', 'AGGGIALPAL'], dtype='<U10'),
 (66461,))

## Compute K-mer of AMR clusters (test)

In [37]:
with open(pantaout_dir + 'samples.json', 'r') as JSON:
    sample_dict = json.load(JSON)
sample2integerindex = {}
for idx in range(len(sample_dict)):
    sample2integerindex[sample_dict[idx]['id']] = idx
n_samples = len(sample_dict)

In [38]:
computed_gene_cluster = [clustersdb2clustersdict[cluster] for cluster in amr_clusterID];

In [39]:
# amr_mat = None;
ksize = 10; # k = 10 for protein, 20 for DNA
kmer_list = [];
pairdata = []
for idx in range(len(computed_gene_cluster)):
    alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    # alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.fna.aln.gz'
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            seqraw = sequence.replace('-', '')
            n_kmers = len(seqraw) - ksize + 1
            for i in range(n_kmers):
                kmer = seqraw[i:i + ksize] 
                kmer_list.append(kmer)
                pairdata.append((sample2integerindex[sample_id], kmer))

In [40]:
unique_kmer = list(set(kmer_list))

In [41]:
kmer2index = {}
for i in range(len(unique_kmer)):
    kmer2index[unique_kmer[i]] = i

In [42]:
kmer_matrix = np.zeros((n_samples, len(unique_kmer)))

In [43]:
# ct = 0
for u, v in pairdata:
    # kmer_matrix[u, kmer2index[v]] += 1
    kmer_matrix[u, kmer2index[v]] = 1

In [44]:
kmer_matrix.shape

(1653, 194690)

In [45]:
kmer_matrixdf = pd.DataFrame(kmer_matrix, columns=unique_kmer)

In [46]:
newkmerindexdb = list(set(kmerindexdb).difference(set(unique_kmer)))

In [47]:
kmer_matrixdf[newkmerindexdb] = 0

In [48]:
kmer_matrixdftest = kmer_matrixdf[kmerindexdb]

In [49]:
kmer_matrixdftest.shape

(1653, 66461)

In [50]:
kmer_matrix_VT = kmer_matrixdftest.values
# np.save(pantaout_dir + 'KmerEncoderAMRGenesSubmission.npy', kmer_matrix_VT) # save numpy array

In [51]:
# np.save(pantaout_dir + 'KmerEncoderAMRGenesSubmission_index.npy', kmerindexdb) # save numpy array

In [52]:
kmer_matrixdftest.head(2)

,NKATLIVEDD,PILMGFWIRG,VAKSLITMVQ,ATEKSMSQIQ,STVTEGALVT,IYPRAQFTKV,LGDSQRAQLV,LQLLSDALGE,MKNLKNRQRA,LSALTASWSR,...,ITLSKEEFEE,QQTMEPGMTV,FNLFIGAFVH,IAPPTITISA,LRGDRALTPC,GVRLPVLNRA,IMTGGGFVGI,GKSLKWGLNH,TANNHADFAR,AGGGIALPAL
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [53]:
kmerindexdb

array(['NKATLIVEDD', 'PILMGFWIRG', 'VAKSLITMVQ', ..., 'GKSLKWGLNH',
       'TANNHADFAR', 'AGGGIALPAL'], dtype='<U10')

## Compute label encoder for core gene clusters (db)

In [54]:
computed_gene_cluster = core_gene_clusterdb;

In [55]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-', 'X']
le = preprocessing.LabelEncoder()
le.fit(codes)

LabelEncoder()

In [56]:
amr_mat = None;
# start_idx = [0];
# pass_gene_cluster = [];
positional_index = [];
# for idx in range(2):
for idx in range(len(computed_gene_cluster)):
    alignment_dir = pantaout_dirdb + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    mat = None; index = 0; index_set = []; selected_location = []
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            if index == 0:
                mat = np.zeros((n_samplesdb, len(sequence)))
            index += 1
            mat[sample2integerindexdb[sample_id],:] = 1 + le.transform([*sequence])
            index_set.append(sample2integerindexdb[sample_id])
            if name == cluster2representativegenedictdb[computed_gene_cluster[idx]]:
                # print(idx, sequence)
                selected_location = [pos for pos, char in enumerate(sequence) if char != '-']
                # print(selected_location)
    mat = mat[:, selected_location] #only select the position where representative sequence is not "-"
    if idx==0:
        amr_mat = mat
        positional_index += [computed_gene_cluster[idx] +'@'+ str(i) for i in range(mat.shape[1])]
    else:
        variant_thres = 0; vs = True;
        if len(index_set) >= int(n_samplesdb*0.01):
            try:
                sel = VarianceThreshold(variant_thres)
                sel.fit(mat[index_set,:])
            except ValueError:
                vs = False
            if vs:
                mat = mat[:, sel.variances_>variant_thres]
                if mat.shape[0] > 0:
                    amr_mat = np.append(amr_mat, mat, axis=1)
                    bool_vec = sel.variances_>variant_thres
                    positional_index += [computed_gene_cluster[idx] +'@'+ str(i) for i in range(len(bool_vec)) if bool_vec[i]]

In [57]:
amr_mat.shape, len(positional_index)

((1248, 556544), 556544)

In [58]:
snpmatrixdbdf = pd.DataFrame(amr_mat, columns = positional_index)

In [59]:
snpmatrixdbdf.head(2)

,groups_0@0,groups_0@1,groups_0@2,groups_0@3,groups_0@4,groups_0@5,groups_0@6,groups_0@7,groups_0@8,groups_0@9,...,valS@901,valS@911,valS@916,valS@917,valS@925,valS@926,valS@931,valS@942,valS@946,valS@947
0,18.0,19.0,9.0,18.0,18.0,2.0,7.0,2.0,2.0,10.0,...,7.0,2.0,19.0,2.0,2.0,10.0,11.0,9.0,2.0,19.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,2.0,19.0,2.0,2.0,10.0,11.0,9.0,2.0,19.0


In [60]:
# metadata_panta = pd.read_csv("data/Kametadata_final.csv")
metadata_panta = pd.read_csv("data/metadata/metadata_final.csv")

In [61]:
metadata_panta.head(2)

,Isolate,Year,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,TBM,TMP,CIP
0,11658_4#1,2006.0,S,S,S,NaN,S,S,R,S,S,S,S,S
1,11657_5#1,2006.0,S,S,R,NaN,R,S,S,S,S,S,R,R


In [62]:
samples_list = list(sample2integerindexdb.keys())

In [63]:
sample_isolate = pd.read_csv('data/metadata/sample_isolate.csv')
sample_isolate.head(2)
sample2isolate = {}
for idx in range(len(sample_isolate.index)):
    sample2isolate[sample_isolate.iloc[idx,0]+'.contig'] = sample_isolate.iloc[idx,1]

In [64]:
# sample2isolate

In [65]:
isolate_list = [sample2isolate[key] for key in samples_list]

In [66]:
metadata_pantanew = metadata_panta.loc[metadata_panta['Isolate'].isin(isolate_list)]

In [67]:
amr_mat = snpmatrixdbdf.values

In [75]:
# Run feature selection
top_features = []
for idx in range(2, metadata_pantanew.shape[1]):
# for idx in range(2, 4):
    y_class = metadata_pantanew.iloc[:,idx].values
    print(metadata_pantanew.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = amr_mat[nonenan_index, ]
    if pa_matrix_new.shape[0] > 0:
        y_new = y[nonenan_index].astype(int)
        scores, pvalue = chi2(pa_matrix_new, y_new)
        # mutual_mat.append(pvalue)
        top_features += list(np.argsort(scores)[::-1][:2000])
top_features.sort()
top_features = list(set(top_features))

CTZ
CTX


In [80]:
kmer_matrix_VT_top_features = amr_mat[:,top_features]
kmer_matrix_VT_top_features.shape

(1248, 2450)

In [81]:
snp_features_name = snpmatrixdbdf.columns[top_features]

In [82]:
snpmatrixdbfinaldf = snpmatrixdbdf.iloc[:,top_features]

## Compute label encoder for core gene clusters (test)

In [83]:
computed_gene_cluster = [clustersdb2clustersdict[cluster] for cluster in core_gene_clusterdb];

In [84]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-', 'X']
le = preprocessing.LabelEncoder()
le.fit(codes)

LabelEncoder()

In [85]:
snp_features_name_unique = np.unique([key.split('@')[0] for key in snp_features_name])
snp_features_name_test_unique =  [clustersdb2clustersdict[cluster] for cluster in snp_features_name_unique];

In [86]:
# amr_mat = None;
amr_mat = np.empty(shape=[n_samples, 0])
positional_index = [];
# for idx in range(100):
for idx in range(len(computed_gene_cluster)):
    if computed_gene_cluster[idx] in snp_features_name_test_unique:
        alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
        mat = None; index = 0; index_set = []; selected_location = []
        with gzip.open(alignment_dir, "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                name, sequence = record.id, str(record.seq)
                sample_id = name.split('-')[0]
                if index == 0:
                    mat = np.zeros((n_samples, len(sequence)))
                index += 1
                mat[sample2integerindex[sample_id],:] = 1 + le.transform([*sequence])
                index_set.append(sample2integerindex[sample_id])
                if name == cluster2representativegenedict[computed_gene_cluster[idx]]:
                    # print(idx, sequence)
                    selected_location = [pos for pos, char in enumerate(sequence) if char != '-']
                    # print(selected_location)
        mat = mat[:, selected_location] #only select the position where representative sequence is not "-"
        positional_index += [core_gene_clusterdb[idx] +'@'+ str(i) for i in range(mat.shape[1])] # use the same name as database
        # if idx==0:
        #     amr_mat = mat
        # else:
        amr_mat = np.append(amr_mat, mat, axis=1)

In [87]:
amr_mat.shape, len(positional_index)

((1653, 219021), 219021)

In [88]:
snpmatrixtestdf = pd.DataFrame(amr_mat, columns = positional_index)

In [89]:
snpmatrixtestfinaldf = snpmatrixtestdf[snp_features_name]

In [99]:
## Run feature selection for AMR Kmer and PA matrix

In [105]:
# Run feature selection (AMR Kmer)
amrkmer_top_features = []
for idx in range(2, metadata_pantanew.shape[1]):
# for idx in range(2, 4):
    y_class = metadata_pantanew.iloc[:,idx].values
    # print(metadata_pantanew.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = kmer_matrixdbdf.iloc[nonenan_index, ]
    if pa_matrix_new.shape[0] > 0:
        y_new = y[nonenan_index].astype(int)
        scores, pvalue = chi2(pa_matrix_new, y_new)
        # mutual_mat.append(pvalue)
        amrkmer_top_features += list(np.argsort(scores)[::-1][:2000])
amrkmer_top_features.sort()
amrkmer_top_features = list(set(amrkmer_top_features))
kmer_matrixdbdf = kmer_matrixdbdf.iloc[:,amrkmer_top_features]
kmer_matrixdftest = kmer_matrixdftest.iloc[:,amrkmer_top_features]

In [110]:
# Run feature selection (PA matrix)
pamatrix_top_features = []
for idx in range(2, metadata_pantanew.shape[1]):
# for idx in range(2, 4):
    y_class = metadata_pantanew.iloc[:,idx].values
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = pa_matrixdb.iloc[nonenan_index, ]
    if pa_matrix_new.shape[0] > 0:
        y_new = y[nonenan_index].astype(int)
        scores, pvalue = chi2(pa_matrix_new, y_new)
        # mutual_mat.append(pvalue)
        pamatrix_top_features += list(np.argsort(scores)[::-1][:2000])
pamatrix_top_features.sort()
pamatrix_top_features = list(set(pamatrix_top_features))
pa_matrixdb = pa_matrixdb.iloc[:,pamatrix_top_features]
pa_matrixnew = pa_matrixnew.iloc[:,pamatrix_top_features]

## Export data

In [119]:
# PA matrix
pa_matrixdb.to_pickle(pantaout_dir + 'PAmatrixTrain.pkl')
pa_matrixnew[~pa_matrixnew.index.isin(pa_matrixdb.index)].to_pickle(pantaout_dir + 'PAmatrixTest.pkl')
# AMR Kmer
kmer_matrixdbdf.index = pa_matrixdb.index
kmer_matrixdftest.index = pa_matrixnew.index
kmer_matrixdbdf.to_pickle(pantaout_dir + 'AMRKmerTrain.pkl')
kmer_matrixdftest[~kmer_matrixdftest.index.isin(kmer_matrixdbdf.index)].to_pickle(pantaout_dir + 'AMRKmerTest.pkl')
# SNPs
snpmatrixdbfinaldf.index = pa_matrixdb.index
snpmatrixtestfinaldf.index = pa_matrixnew.index
snpmatrixdbfinaldf.to_pickle(pantaout_dir + 'SNPmatrixTrain.pkl')
snpmatrixtestfinaldf[~snpmatrixtestfinaldf.index.isin(snpmatrixdbfinaldf.index)].to_pickle(pantaout_dir + 'SNPmatrixTest.pkl')